In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Bhagvad Gita.csv')

In [3]:
df.head()

,Chapter No,Verse No,Shloka,English Translation,Explanation
0,1,1,धृतराष्ट्र उवाच | धर्मक्षेत्रे कुरुक्षेत्रे सम...,"Dhritarastra said: O Sanjaya, what did my sons...",The two armies had gathered on the battlefield...
1,1,2,सञ्जय उवाच । दृष्ट्वा तु पाण्डवानीकं व्यूढं दु...,"Sanjaya said: But then, seeing the army of the...","Sanjay understood Dhritarashtra’s concern, who..."
2,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् । व...,"O teacher, (please) see this vast army of the ...",Duryodhana asked Dronacharya to look at the sk...
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि | युयुधा...,"There are in this army, heroes wielding great ...","Due to his anxiety, the Pandava army seemed mu..."
4,1,5,धृष्टकेतुश्चेकितान: काशिराजश्च वीर्यवान् | पुर...,"Dhrstaketu, Cekitana, and the valiant king of ...","Due to his anxiety, the Pandava army seemed mu..."


In [4]:
#CREATE TABLE bhagvadgita(bhagvadgita_id serial primary key, chapter_no INT(64), verse_no INT(64), shloka TEXT, english_translation TEXT, explanation TEXT);

In [5]:
import pandas as pd
import psycopg2

import yaml
import warnings

warnings.filterwarnings('ignore')

try:
    with open('connection.yaml', 'r') as file:
        infos = yaml.safe_load(file) 


    conn = psycopg2.connect(user= infos['user'],
                        password= infos['password'],
                        host= infos['host'],
                        database= infos['database'])


    print("Connected to the PostgreSQL database.")

except psycopg2.Error as e:
    print(f"Unable to connect to the postgresql database. Error:{e} ")



Connected to the PostgreSQL database.


In [6]:
def read_bhagvadgita(query):
    df = pd.read_sql(query, conn)
    return df

In [7]:
read_bhagvadgita('SELECT * FROM bhagvadgita')

,bhagvadgita_id,chapter_no,verse_no,shloka,english_translation,explanation


In [8]:
insert_query = """INSERT INTO bhagvadgita (chapter_no, verse_no, shloka, english_translation, explanation) VALUES (%s, %s, %s, %s, %s)"""

In [9]:

from psycopg2.extensions import register_adapter, AsIs
import numpy as np
register_adapter(np.int64,AsIs)

def insert_bhagvadgita_many(df):
    

    conn = psycopg2.connect(user= infos['user'],
                        password= infos['password'],
                        host= infos['host'],
                        database= infos['database'])
    
    cursor = conn.cursor()
 
    for i in range(len(df)):
        cursor.execute(insert_query, tuple(df.iloc[i, :]))
        if i % 700 == 0:
            conn.commit()

        conn.commit()
    cursor.close()
    print(f'{len(df)} rows were inserted.')
    

In [10]:
insert_bhagvadgita_many(df)

700 rows were inserted.


In [11]:
read_bhagvadgita ('SELECT * FROM BHAGVADGITA')

,bhagvadgita_id,chapter_no,verse_no,shloka,english_translation,explanation
0,1,1,1,धृतराष्ट्र उवाच | धर्मक्षेत्रे कुरुक्षेत्रे सम...,"Dhritarastra said: O Sanjaya, what did my sons...",The two armies had gathered on the battlefield...
1,2,1,2,सञ्जय उवाच । दृष्ट्वा तु पाण्डवानीकं व्यूढं दु...,"Sanjaya said: But then, seeing the army of the...","Sanjay understood Dhritarashtra’s concern, who..."
2,3,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् । व...,"O teacher, (please) see this vast army of the ...",Duryodhana asked Dronacharya to look at the sk...
3,4,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि | युयुधा...,"There are in this army, heroes wielding great ...","Due to his anxiety, the Pandava army seemed mu..."
4,5,1,5,धृष्टकेतुश्चेकितान: काशिराजश्च वीर्यवान् | पुर...,"Dhrstaketu, Cekitana, and the valiant king of ...","Due to his anxiety, the Pandava army seemed mu..."
...,...,...,...,...,...,...
695,696,18,74,सञ्जय उवाच | इत्यहं वासुदेवस्य पार्थस्य च महात...,Sanjaya said: I thus heard this conversation o...,"In this way, Sanjay comes to the end of his na..."
696,697,18,75,व्यासप्रसादाच्छ्रुतवानेतद्गुह्यमहं परम् | योगं...,Through the favour of Vyasa I heard this secre...,"Shree Krishna Dwaipayan Vyasadev, also known a..."
697,698,18,76,राजन्संस्मृत्य संस्मृत्य संवादमिममद्भुतम् | के...,"And, O king, while repeatedly remembering this...",A spiritual experience gives a happiness that ...
698,699,18,77,तच्च संस्मृत्य संस्मृत्य रूपमत्यद्भुतं हरे: | ...,"O king, repeatedly recollecting that greatly e...",Arjun was blessed with the vision of the cosmi...
